### 2015-02 2do Recuperatorio
Se tiene un RDD con libros en donde cada registro es un texto. Se pide obtener todos los anagramas de mas de 7 letras que puedan encontrarse. El formato de salida debe ser una lista de listas en donde cada lista tiene un conjunto de palabras que son anagramas. Ejemplo: [[discounter,introduces,reductions],[percussion,supersonic]...]

In [34]:
textFile = sc.textFile("data/texts.txt")
textFile.take(2)

[u'Lorem ascent ipsum dolor sit amet, autem melius ea pri, pro malis theophrastus ut, in duo vide vocibus instructior. Duo denique torquatos dissentiunt ei, cu reprimique interesset mei. Duo nulla paulo ea. Ex commodo convenire gubergren mea, verterem lobortis ut mea. Qui debet ocurreret hendrerit ea, iracundia rationibus cotidieque et sea. recontrarepetida recontrarepetida recontrarepetida recontrarepetida',
 u'Ius secant nibh utinam ut. Vel mollis integre labores an. Qualisque incorrupte mea ut, labitur sententiae theophrastus est ea. Usu in alterum rationibus, vix dico prodesset ex, suas offendit postulant  recontrarepetida recontrarepetida recontrarepetida ne eum. Has causae corrumpit te.']

In [49]:
words = textFile.flatMap(lambda line: line.split())
words.take(5)

[u'Lorem', u'ascent', u'ipsum', u'dolor', u'sit']

In [50]:
words.filter(lambda x: len(x) > 7)\
     .distinct()\
     .map(lambda x:(''.join(sorted(x)), [x]))\
     .reduceByKey(lambda a, b: a + b)\
     .filter(lambda x: len(x[1]) > 1)\
     .map(lambda x: x[1])\
     .collect()

[[u'doctorwho', u'torchwood']]

---
### 2015-01 1er Parcial
Dada una colección de documentos queremos encontrar frases de 1 , 2 o 3 palabras que sean anagramas de otras. Por ejemplo: (“Postmaster”, “Stamp store”) o (“A telescope” , “To see Place”) o (“The cockroach”, “cook catch her”). Esta tarea implica una combinatoria muy difícil por lo que se decide usar Map-Reduce para paralelizarla. Usando Map-Reduce programar la solución a este problema listando todos los pares de anagramas entre frases de 1, 2 o 3 palabras. Como puede verse en los ejemplos se ignoran mayúsculas y minúsculas y los espacios en blanco, puntuación, etc. Suponer que existe la función word_tokenizer que recibe un texto y devuelve un vector de palabras ya convertidas a minúsculas y sin puntuación. 

In [1]:
import re

def tokenizer(str, pipeline):
    if len(pipeline) == 0:
        return str
    else:
        return tokenizer(pipeline[0](str), pipeline[1:])
    
def word_tokenizer(doc):
    pipeline = [ lambda s: re.sub('[^\w\s]', '', s), 
            lambda s: re.sub('[\d]', '', s),
            lambda s: s.lower() ]
    
    return tokenizer(doc, pipeline).split()

In [67]:
def biphrases(t):
    return [ ' '.join(t[i:i + 2]) for i in range(0, len(t) - 1)]

def triphrases(t):
    return [ ' '.join(t[i:i + 3]) for i in range(0, len(t) - 2)]

In [75]:
phrases = sc.textFile("data/texts.txt")\
             .map(lambda x: word_tokenizer(x))\
             .flatMap(lambda x: triphrases(x) + biphrases(x) + x)\
             .distinct()\
             .map(lambda x: (''.join(sorted(''.join(x).replace(' ',''))), [x]))\
             .reduceByKey(lambda a, b: a + b)\
             .filter(lambda x: len(x[1]) > 1)\
             .map(lambda x: tuple(x[1]))
phrases.collect()

[(u'ascent', u'secant', u'stance'),
 (u'mea', u'eam'),
 (u'postmaster', u'stamp store'),
 (u'cu ei', u'ei cu'),
 (u'doctorwho', u'torchwood'),
 (u'ut mea', u'mea ut', u'autem'),
 (u'te', u'et'),
 (u'eum in', u'in eum'),
 (u'et sea', u'est ea')]